## There are 4 locations (labeled by different letters), and our job is to pick up the passenger at one location and drop him off at another. We receive +20 points for a successful drop-off and lose 1 point for every time-step it takes. There is also a 10 point penalty for illegal pick-up and drop-off actions.

### Installing gym

In [ ]:
!pip install cmake 'gym[atari]' scipy

### Seeing how it looks like using render

In [ ]:
import gym
env = gym.make("Taxi-v3").env # We are using the .env on the end of make to avoid training stopping at 200 iterations
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [ ]:
env.reset() # reset environment to a new, random state
env.render()
print("Action Space {}".format(env.action_space)) 
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


#### The filled square represents the taxi, which is yellow without a passenger and green with a passenger.
#### The pipe ("|") represents a wall which the taxi cannot cross.
#### R, G, Y, B are the possible pickup and destination locations. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

### As seen above we have an Action Space of size 6 and a State Space of size 500.
#### Action number from 0-5 represents:
  0 = south

  1 = north
  
  2 = east
  
  3 = west
  
  4 = pickup
  
  5 = dropoff

### We can encode the environment according to our wish too

In [ ]:
state = env.encode(2,1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)
env.s = state
env.render()  

State: 228
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


In [ ]:
env.P[228] # Reward table illustration

{0: [(1.0, 328, -1, False)],
 1: [(1.0, 128, -1, False)],
 2: [(1.0, 248, -1, False)],
 3: [(1.0, 208, -1, False)],
 4: [(1.0, 228, -10, False)],
 5: [(1.0, 228, -10, False)]}

#### The dictionary above has the structure as : { action: [ ( probability, nextstate, reward, done) ] }

### Initializing the Q-table to a 500×6 matrix of zeros:

In [ ]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

### Training the agent

In [ ]:
%%time

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 59.1 s, sys: 9.51 s, total: 1min 8s
Wall time: 1min


#### Checking what the Q-values are at our illustration's state which we assumed above



In [ ]:
q_table[228]

array([ -2.36395049,  -2.36395106,  -2.36395034,  -2.1220864 ,
       -11.27324419, -11.27314499])

The max Q-value is "north" (-2.27)



### Evaluate agent's performance after Q-learning

In [ ]:
total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.42
Average penalties per episode: 0.0


Notebook Link - https://colab.research.google.com/drive/1vmzDcji417m4-pfCIHc2MvoOnL7LKhkw?usp=sharing